In [1]:
from stringprep import b1_set

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open('data/names.txt', 'r') as f:
    words = f.read().splitlines()
print(words[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(stoi)

In [4]:
block_size = 3 # context length

def build_dataset(words):
    # build the dataset
    X, Y = [], []
    for w in words:
        # print(w)
        # Лист болванка из 0 для дальнейшего заполнения
        context = [0] * block_size
        # print(context)
        for ch in w + '.':
            # Индекс символа в словаре
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)

            # print(''.join(itos[i] for i in context), '------>', itos[ix])
            # Убрали первый символ, добавили новый
            context = context[1:] + [ix] # crop and append
            # print(context)

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

In [5]:
import random

# Начало случайного распределения зафиксировано, дальше все что будет генерироваться при перезапуске программы будет одинаковым. Если вызывать несколько раз внутри одной и той же программы, результаты будут разными
random.seed(42)

random.shuffle(words)

n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [6]:
# Функция которая будет использоваться в последствии для сравнения градиентов расчитанных вручную с градиентами pytorch
def cmp(s, dt, t):
    '''


    :param s: Название того что сверяем
    :param dt: градиент расчитанный вручную
    :param t: Значения для которых pytorch вычислит градиент
    :return:
    '''
    # Проверка точного совпадения значений
    ex = torch.all(dt == t.grad).item()
    # Проверка приблизительного совпадения значений
    app = torch.allclose(dt, t.grad)
    # Максимальная существующая ошибка
    maxdiff = (dt - t.grad).abs().max().item()
    return print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [7]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [8]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

### forward pass, "chunkated" into smaller steps that are possible to backward one at a time

In [9]:
emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors

In [10]:
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation

In [11]:
# BatchNorm layer
bnmeani = 1/n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

In [12]:
# Non-linearity
h = torch.tanh(hpreact) # hidden layer

In [13]:
# Linear layer 2
logits = h @ W2 + b2 # output layer

cross entropy loss (same as F.cross_entropy(logits, Yb))

In [14]:
# Максимальный логит для каждого примера
logit_maxes = logits.max(1, keepdim=True).values
# Сделано для безопасности, чтобы не возводить в слишком большие степени
norm_logits = logits - logit_maxes

# Число e возводится в степень norm_logits
# То же самое что и counts = torch.exp(norm_logits)
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)

# 1 / counts_sum
counts_sum_inv = counts_sum ** -1

# Softmax: преобразование логитов в вероятности
# значения ∈ (0, 1)
# сумма по классам = 1
# Тут закопан broadcasting, вектор counts_sum_inv был растянут по колонкам
probs = counts * counts_sum_inv

logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

In [15]:
# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3426, grad_fn=<NegBackward0>)

### 1) dlogprobs

loss = -logprobs[range(n), Yb].mean()

In [16]:
# Вероятности каждого из 27 возможных символов для каждого из 32 символов в батче
logprobs.shape

torch.Size([32, 27])

In [17]:
# Верные ответы
Yb

tensor([ 8, 14, 15, 22,  0, 19,  9, 14,  5,  1, 20,  3,  8, 14, 12,  0, 11,  0,
        26,  9, 25,  0,  1,  1,  7, 18,  9,  3,  5,  9,  0, 18])

In [18]:
Yb.shape

torch.Size([32])

In [19]:
# Вероятность данная верным символам
logprobs[range(n), Yb]

tensor([-3.9667, -3.0268, -3.5204, -3.3059, -4.1932, -3.4708, -3.2121, -4.0400,
        -3.2103, -4.2376, -3.1360, -1.6506, -2.8540, -2.9172, -2.9967, -3.2098,
        -3.7158, -3.0043, -3.5757, -3.4735, -2.9658, -2.9524, -4.3527, -4.1069,
        -3.3889, -2.8605, -3.0164, -3.9435, -2.7338, -3.5007, -3.3282, -3.0970],
       grad_fn=<IndexBackward0>)

In [20]:
logprobs[range(n), Yb].shape

torch.Size([32])

Итого для каждого из 32 примеров в batch у нас есть вероятность для каждого из 27 потенциальных символов. Для верного ответа считается градиент ошибки, для остальных 26 нет

In [21]:
# dlogprobs = torch.zeros((logprobs.shape))
dlogprobs = torch.zeros_like(logprobs)
dlogprobs.shape

torch.Size([32, 27])

In [22]:
# dloss = - 1/n (n=32 в нашем случае)
dlogprobs[range(n), Yb] = -1.0 / n

In [23]:
# Сверка корректности расчета с torch
cmp('logprobs', dlogprobs, logprobs)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


### 2) dprobs

logprobs = probs.log()

В PyTorch torch.log(x) — это натуральный логарифм \ln(x).

![lnderin](images/lnderiv.png)

In [24]:
logprobs.shape

torch.Size([32, 27])

In [25]:
probs.shape

torch.Size([32, 27])

In [26]:
dprobs = torch.zeros_like(logprobs)

In [27]:
# x = probs
# (1/ probs) - текущая производная
# dlogprobs - производная предыдущего слоя
dprobs = (1/ probs) * dlogprobs

In [28]:
# Сверка корректности расчета с torch
cmp('dprobs', dprobs, probs)

dprobs          | exact: True  | approximate: True  | maxdiff: 0.0


### 3) dcounts_sum_inv

Важное замечание, когда я считаю производную функции, я считаю производную строки где она была примененена, а не где она была вычислена, то есть не counts_sum_inv = counts_sum ** -1, а probs = counts * counts_sum_inv



probs = counts * counts_sum_inv

In [29]:
# Тут надо быть аккуратнее, имел место broadcasting
counts.shape, counts_sum_inv.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [30]:
dprobs.shape

torch.Size([32, 27])

In [31]:
# Форма предыдущего слоя
dcounts_sum_inv = torch.zeros_like(counts)
dcounts_sum_inv

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
      

Ключевая идея (сразу)

.sum() нужен ТОЛЬКО тогда, когда один и тот же элемент входа
повлиял на НЕСКОЛЬКО элементов выхода.

Если влияние 1 → 1, суммировать нечего.

In [32]:
# x = counts
# dprobs = градиент предыдущего слоя probs

# counts * dprobs = x * производная предыдущего слоя, .sum(1, keepdim=True) - учет broadcasting
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)

In [33]:
counts.shape, counts_sum_inv.shape, dcounts_sum_inv.shape

(torch.Size([32, 27]), torch.Size([32, 1]), torch.Size([32, 1]))

In [34]:
cmp('dcounts_sum_inv', dcounts_sum_inv, counts_sum_inv)

dcounts_sum_inv | exact: True  | approximate: True  | maxdiff: 0.0


### 4) dcounts1

In [35]:
# Здесть почему-то не надо делать .sum()
dcounts = counts_sum_inv * dprobs
# cmp('dcounts', dcounts, counts)

# Пока что нет возможности проверить корректность counts тк оно влияет на результат не только в этом месте, а еще и в counts_sum = counts.sum(1, keepdims=True)

In [36]:
counts_sum_inv.shape, dprobs.shape

(torch.Size([32, 1]), torch.Size([32, 27]))

### 5) dcounts_sum

counts_sum_inv = counts_sum ** -1

<img src="images/power-rule-formula.png" width="400">

In [37]:
# n = -1, x = counts_sum
# dcounts_sum_inv - градиент предыдущего слоя

dcounts_sum = (-1 * counts_sum **-2) * dcounts_sum_inv
cmp('dpow', dcounts_sum, counts_sum)

dpow            | exact: True  | approximate: True  | maxdiff: 0.0


### 6) Dcounts2

counts_sum = counts.sum(1, keepdims=True)


In [38]:
# a11 a12 a13  --> b1(a11 + a12 + a13)
# a21 a22 a23  --> b1(a31 + a22 + a23)
# s31 a32 a33  --> b1(a31 + a32 + a33)

In [39]:
counts.shape, counts_sum.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [40]:
# В операции .sum градиенты просто перетекают в след слой, поэтому я растянул их согласно размерностям
# torch.broadcast_to(dcounts, counts.shape)
# dcounts состоит из двух операций, первую мы посчитали выше, теперь прибавляю к ней результат текущей, и делаю проверку

dcounts += torch.broadcast_to(dcounts, counts.shape)
dcounts_sum.shape

torch.Size([32, 1])

In [41]:
cmp('dcounts_sum', dcounts_sum, counts_sum)

dcounts_sum     | exact: True  | approximate: True  | maxdiff: 0.0
